In [ ]:
#Link to dataset : https://www.kaggle.com/competitions/tensorflow-speech-recognition-challenge/data

import zipfile

!unzip '/content/drive/MyDrive/speech_commands.zip' -d '/content/drive/MyDrive/speech_commands'

Streaming output truncated to the last 5000 lines.
  inflating: /content/drive/MyDrive/speech_commands/speech_commands/four/c0fb6812_nohash_0.wav  
  inflating: /content/drive/MyDrive/speech_commands/__MACOSX/speech_commands/four/._c0fb6812_nohash_0.wav  
  inflating: /content/drive/MyDrive/speech_commands/speech_commands/four/35d1b6ee_nohash_0.wav  
  inflating: /content/drive/MyDrive/speech_commands/__MACOSX/speech_commands/four/._35d1b6ee_nohash_0.wav  
  inflating: /content/drive/MyDrive/speech_commands/speech_commands/four/1496195a_nohash_1.wav  
  inflating: /content/drive/MyDrive/speech_commands/__MACOSX/speech_commands/four/._1496195a_nohash_1.wav  
  inflating: /content/drive/MyDrive/speech_commands/speech_commands/four/ceef6d96_nohash_2.wav  
  inflating: /content/drive/MyDrive/speech_commands/__MACOSX/speech_commands/four/._ceef6d96_nohash_2.wav  
  inflating: /content/drive/MyDrive/speech_commands/speech_commands/four/37fc5d97_nohash_0.wav  
  inflating: /content/drive/MyDr

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow
print(tensorflow.__version__)


1.15.2


In [ ]:
pip install python_speech_features

In [ ]:
import tensorflow.contrib.slim as slim

In [ ]:
from tqdm import tqdm_notebook as tqdm
import time
import os
import librosa
from python_speech_features import mfcc
from tensorflow.contrib import rnn
import tensorflow as tf
import numpy as np
import keras
from sklearn.metrics import confusion_matrix

In [ ]:
audio_path = '/content/drive/MyDrive/speech_commands/speech_commands/'

In [ ]:
all_wave = []
for folder in os.listdir(audio_path):
    folder_path = os.path.join(audio_path, folder)
    if (os.path.isdir(folder_path) and folder!='_background_noise_'):
        word_path = os.path.join(audio_path, folder)
        for audio_file in os.listdir(word_path):
            all_wave.append(audio_path + folder + '/' + audio_file)

In [ ]:
#reading the validation list
validation_audio = []
with open(audio_path + "validation_list.txt", "r") as f:
    for line in f:
        validation_audio.append(audio_path+line)
        
#reading the test list
testing_audio = []
with open(audio_path + "testing_list.txt", "r") as g:
    for line in g:
        testing_audio.append(audio_path+line)

print(validation_audio)
print(testing_audio)

['/content/drive/MyDrive/speech_commands/speech_commands/right/a69b9b3e_nohash_0.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/439c84f4_nohash_1.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/409c962a_nohash_1.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/dbaf8fc6_nohash_2.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/a6d586b7_nohash_1.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/0e17f595_nohash_1.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/ff2b842e_nohash_3.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/7fd25f7c_nohash_2.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/3cc595de_nohash_4.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/591d32f3_nohash_0.wav\n', '/content/drive/MyDrive/speech_commands/speech_commands/right/52bfbce8_nohash_0.wav\n', '/content/drive/MyDrive/speech_

In [ ]:
join_lists = []
join_lists = validation_audio + testing_audio
t = map(lambda s: s.strip(), join_lists)
training_audio=[]
training_audio = [x for x in all_wave if x not in t]
print(len(training_audio))
print(training_audio[:10])

105829
['/content/drive/MyDrive/speech_commands/speech_commands/right/8e523821_nohash_2.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/bb05582b_nohash_3.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/988e2f9a_nohash_0.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/a69b9b3e_nohash_0.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/1eddce1d_nohash_3.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/93ec8b84_nohash_0.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/7e7ca854_nohash_0.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/6272b231_nohash_1.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/439c84f4_nohash_1.wav', '/content/drive/MyDrive/speech_commands/speech_commands/right/2f813234_nohash_1.wav']


In [ ]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

In [ ]:
labels = ['yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']

#Extracting features using MFCC on Training data
mfcc_training = []
training_labels = [] 
for wav in tqdm(training_audio):
    samples, sample_rate = librosa.load(wav, sr = 16000)
    mfccs = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
    a = os.path.basename(os.path.dirname(wav))
    if(len(samples) == 16000):
        if a in labels:
            mfcc_training.append(mfccs)
            training_labels.append(a)
        else:
            mfcc_training.append(mfccs)
            training_labels.append('unknown')


#Extracting features using MFCC on Validation data
mfcc_valid = []
valid_labels = []
c = map(lambda s: s.strip(), validation_audio)
for wav in tqdm(c):
    samples, sample_rate = librosa.load(wav, sr = 16000)
    mfccs = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
    a = os.path.basename(os.path.dirname(wav))
    if(len(samples) == 16000):
        if a in labels:
            mfcc_valid.append(mfccs)
            valid_labels.append(a)
        else:
            mfcc_valid.append(mfccs)
            valid_labels.append('unknown')

#Extracting features using MFCC on Testing data
mfcc_test = []
test_labels = []
d = map(lambda s: s.strip(), testing_audio)
for wav in tqdm(d):
    samples, sample_rate = librosa.load(wav, sr = 16000)
    mfccs = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
    a = os.path.basename(os.path.dirname(wav))
    if(len(samples) == 16000):
        if a in labels:
            mfcc_test.append(mfccs)
            test_labels.append(a)
        else:
            mfcc_test.append(mfccs)
            test_labels.append('unknown')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/105829 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(training_labels)
y_valid = le.fit_transform(valid_labels)
y_test = le.fit_transform(test_labels)
classes = list(le.classes_)
len(classes)

11

In [ ]:
from keras.utils import np_utils
y_train=np_utils.to_categorical(y_train, num_classes=len(classes))
y_valid=np_utils.to_categorical(y_valid, num_classes=len(classes))
y_test=np_utils.to_categorical(y_test, num_classes=len(classes))

In [ ]:
mfcc_train = np.array(mfcc_training)
mfcc_valid = np.array(mfcc_valid)
mfcc_test = np.array(mfcc_test)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

# Feature Scaling on training dataset
num_instances, num_time_steps, num_features = mfcc_train.shape
mfcc_train = np.reshape(mfcc_train, newshape=(-1, num_features))
mfcc_train = sc.fit_transform(mfcc_train)
mfcc_train = np.reshape(mfcc_train, newshape=(num_instances, num_time_steps, num_features))

# Feature Scaling on validation dataset
num_instances, num_time_steps, num_features = mfcc_valid.shape
mfcc_valid = np.reshape(mfcc_valid, newshape=(-1, num_features))
mfcc_valid = sc.transform(mfcc_valid)
mfcc_valid = np.reshape(mfcc_valid, newshape=(num_instances, num_time_steps, num_features))

# Feature Scaling on testing dataset
num_instances, num_time_steps, num_features = mfcc_test.shape
mfcc_test = np.reshape(mfcc_test, newshape=(-1, num_features))
mfcc_test = sc.fit_transform(mfcc_test)
mfcc_test = np.reshape(mfcc_test, newshape=(num_instances, num_time_steps, num_features))

In [ ]:
from tensorflow.contrib import rnn

In [ ]:
n_input = mfcc_train.shape[2] 
n_steps = mfcc_train.shape[1] 
n_hidden = 64
n_classes = len(classes)
batch_size = 100
num_epochs = 100
no_of_batches = 100

In [ ]:
x = tf.placeholder(tf.float32, [None, n_steps, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
weights = {
   'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
   'out': tf.Variable(tf.random_normal([n_classes]))
}
keep_prob = tf.placeholder(tf.float32)


In [ ]:
def next_batch(num, data, labels):
    
    idx = np.arange(0 , len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[ i] for i in idx]
    labels_shuffle = [labels[ i] for i in idx]

    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [ ]:
#Define RNN  
def RNN(x, weights, biases):
    x = tf.unstack(x, n_steps, 1)

   # Defining the lstm cell 
    lstm_cell = rnn.BasicLSTMCell(n_hidden, forget_bias = 0.2)
    
   # Getting the output of the lstm cell
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype = tf.float32)

   # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

In [ ]:
#prediction
pred = RNN(x, weights, biases)

# Defining loss and optimizer with learning rate
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(cost)

# Evaluating model
correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [ ]:
with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        loss_train = 0;
        loss_valid = 0;
        for i in range(no_of_batches):  
            batch_x, batch_y = next_batch(batch_size, mfcc_train, y_train)
            batch_x_val, batch_y_val = next_batch(batch_size, mfcc_valid, y_valid)
            batch_x = batch_x.reshape((batch_size, n_steps, n_input))
            batch_x_val = batch_x_val.reshape((batch_size, n_steps, n_input))
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            training_acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
            valid_acc = sess.run(accuracy, feed_dict={x: batch_x_val, y: batch_y_val})
            loss_train += sess.run(cost, feed_dict={x: batch_x, y: batch_y})
            loss_valid += sess.run(cost, feed_dict={x: batch_x_val, y: batch_y_val})
        avg_loss_training = loss_train / no_of_batches
        avg_loss_validation = loss_valid / no_of_batches
        print("Epoch " + str(epoch) + ", Training Loss= " + \
              "{:.6f}".format(avg_loss_training) + ", Training Accuracy= " + \
              "{:.5f}".format(training_acc) + ", Validation Loss= " + \
              "{:.5f}".format(avg_loss_validation) + ", Validation Accuracy= " + \
              "{:.5f}".format(valid_acc))
    saver.save(sess, 'content/the_test_model')   
    print("Finished Optimization process")
    validate_feed = {x: mfcc_valid, y: y_valid}
    print("Validation Accuracy:", \
        sess.run(accuracy, feed_dict=validate_feed))
    test_feed = {x:mfcc_test , y: y_test}
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict=test_feed))

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device

Epoch 0, Training Loss= 1.731564, Training Accuracy= 0.64000, Validation Loss= 1.71956, Validation Accuracy= 0.67000
Epoch 1, Training Loss= 1.497226, Training Accuracy= 0.59000, Validation Loss= 1.47631, Validation Accuracy= 0.63000
Epoch 2, Training Loss= 1.416040, Training Accuracy= 0.58000, Validation Loss= 1.43946, Validation Accuracy= 0.55000
Epoch 3, Training Loss= 1.360217, Training Accuracy= 0.62000, Validation Loss= 1.39538, Validation Accuracy= 0.66000
Epoch 4, Training Loss= 1.391903, Training Accuracy= 0.57000, Validation Loss= 1.38849, Validation Accuracy= 0.61000
Epoch 5, Training Loss= 1.319126, Training Accuracy= 0.63000, Validation Loss= 1.34034, Validation Accuracy= 0.53000
Epoch 6, Training Loss= 1.323395, Training Accuracy= 0.61000, Validation Loss= 1.33497, Validation Accuracy= 0.67000
Epoch 7, Training Loss= 1.273281, Training Accuracy= 0.72000, Validation Loss= 1.27971, V

In [ ]:
#Make a single prediction

#importing the file
training_audio_path = '/content/drive/MyDrive/Colab Notebooks/train/audio/bird/'
samples, sample_rate = librosa.load(training_audio_path+'1a9afd33_nohash_1.wav', sr = 16000)
#geting mfccs
mfcc1 = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc1 = mfcc1[np.newaxis, :, :]
#normalization of data
num_instances1, num_time_steps1, num_features1 = mfcc1.shape
mfcc1 = np.reshape(mfcc1, newshape=(-1, num_features1))
mfcc1 = sc.fit_transform(mfcc1)
mfcc1 = np.reshape(mfcc1, newshape=(num_instances1, num_time_steps1, num_features1))

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#Single prediction
#importing file
train_audio_path = '/content/drive/MyDrive/Colab Notebooks/train/audio/bird/'
samples, sample_rate = librosa.load(train_audio_path+'0a7c2a8d_nohash_0.wav', sr = 16000)
#geting mfccs
mfcc2 = mfcc(samples, samplerate=sample_rate, numcep=13, nfilt=26, nfft = 512)
mfcc2 = mfcc2[np.newaxis, :, :]
#normalization of data
num_instances2, num_time_steps2, num_features2 = mfcc2.shape
mfcc2 = np.reshape(mfcc2, newshape=(-1, num_features2))
mfcc2 = sc.fit_transform(mfcc2)
mfcc2 = np.reshape(mfcc2, newshape=(num_instances2, num_time_steps2, num_features2))